# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [17]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
import time
import os
from torchsummary import summary

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [7]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [49]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(     #input_size=(1*28*28)
            nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,stride=1,padding=2),  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2 ,stride = 2) 
        )
 
        self.conv2 = nn.Sequential(
            nn.Conv2d(6,16,5,1,0),                      #input_size=(6*14*14)，output_size=16*10*10
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2,stride = 2)    ##input_size=(16*10*10)，output_size=(16*5*5)
        )
 
        self.fc1 = nn.Sequential(
            nn.Linear(16*5*5,160),
            nn.ReLU()
        )
 
        self.fc2 = nn.Sequential(
            nn.Linear(160,84),
            nn.ReLU()
        )
 
        self.fc3 = nn.Linear(84,10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
 
        x = x.view(x.size(0), -1) #全连接层,数据拉为一维
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
model = SimpleNet()
best_loss = 100
summary(model,(1,28,28))
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(),lr = 0.001,momentum = 0.9)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              ReLU-2            [-1, 6, 28, 28]               0
         MaxPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              ReLU-5           [-1, 16, 10, 10]               0
         MaxPool2d-6             [-1, 16, 5, 5]               0
            Linear-7                  [-1, 160]          64,160
              ReLU-8                  [-1, 160]               0
            Linear-9                   [-1, 84]          13,524
             ReLU-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
Total params: 81,106
Trainable params: 81,106
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

Next, we can start to train and evaluate!

In [47]:
# train and evaluate
loss_sum = 0
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        loss_sum+=loss.item()
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    loss_sum = loss_sum/len(train_loader)
    if loss_sum<best_loss:
        best_loss = loss_sum
        torch.save(model.state_dict(),'MNIST.pth')
    print('[Epoch:{}] train loss:{}'.format(epoch,loss_sum))
    loss_sum = 0

100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:44<00:00, 10.60it/s]


[Epoch:0] train loss:2.287298430234958


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:40<00:00, 11.42it/s]


[Epoch:1] train loss:1.7480472024434652


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:39<00:00, 11.96it/s]


...

[Epoch:7] train loss:0.10834409459693055


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:42<00:00, 11.01it/s]


[Epoch:8] train loss:0.09670634345254964


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:41<00:00, 11.17it/s]


[Epoch:9] train loss:0.08682902615405938


#### Q5:
Please print the training and testing accuracy.

In [48]:
correct = 0
total = 0
for test_inputs, labels in tqdm(test_loader):
    outputs_test = model(test_inputs)
    _, predicted = torch.max(outputs_test.data, 1)  #输出得分最高的类
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('测试集准确率为：{}%'.format((100.0 * correct / total)))
correct = 0
total = 0
for test_inputs, labels in tqdm(train_loader):
    outputs_test = model(test_inputs)
    _, predicted = torch.max(outputs_test.data, 1)  #输出得分最高的类
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('训练集集准确率为：{}%'.format((100.0 * correct / total)))

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:03<00:00, 23.18it/s]


测试集准确率为：97.4959945678711%


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:19<00:00, 23.62it/s]


训练集集准确率为：97.42755126953125%
